In [1]:
import fastfusion
import fastfusion as ff
from fastfusion.util import set_n_parallel_jobs
from fastfusion.mapper import Metrics
import time

set_n_parallel_jobs(12)

from fastfusion.mapper.FFM._join_pmappings.sim import SIM

# Set "MainMemory" to whatever the off-chip memory is
def no_hybrid_mapping_filter(pm: SIM):
    fused_tensors = [x for x in pm.compatibility.tensors if x.resource_name != "MainMemory"]
    # Same # of fused loops above all fused tensors
    return len(set(len(x.loops) for x in fused_tensors)) <= 1

def no_fused_filter(pm: SIM):
    return all(reservation.resource_name == "MainMemory" for reservation in pm.compatibility.tensors)

import csv
import pickle

from IPython.display import display, SVG

from paths import ARCH_DIR, WORKLOADS_DIR

In [ ]:
def make_spec(arch_fname, workload_fname):
    spec = fastfusion.Specification.from_yaml(
        ARCH_DIR / arch_fname,
        WORKLOADS_DIR / workload_fname,
    )
    spec.mapper.ffm.metrics = Metrics.LATENCY | Metrics.ENERGY
    # spec.mapper.ffm.max_fused_loops = 3
    spec.mapper.ffm.max_fused_loops_per_rank_variable = 1
    return spec

def run_make_pmappings(spec):
    start = time.time()
    pmappings = fastfusion.mapper.FFM.make_pmappings(spec)
    end = time.time()
    duration = end - start
    return pmappings, duration

def run_join_pmappings(spec, pmappings):
    start = time.time()
    mappings = fastfusion.mapper.FFM.join_pmappings(spec, pmappings)
    end = time.time()
    duration = end - start
    return mappings, duration

def run_experiments(shape):
    spec = make_spec("nvdla.arch.yaml", f"mobilenet_{shape}.yaml")

    # LoopForest
    pmappings = ff.mapper.FFM.make_pmappings(spec)
    loopforest_mappings = ff.mapper.FFM.join_pmappings(spec, pmappings)

    # LoopTree
    lt_pmappings = pmappings.filter(no_hybrid_mapping_filter)
    looptree_mappings = ff.mapper.FFM.join_pmappings(spec, lt_pmappings)

    # Unfused
    un_pmappings = pmappings.filter(no_fused_filter)
    unfused_mappings = ff.mapper.FFM.join_pmappings(spec, un_pmappings)

    # TileFlow
    spec.mapper.ffm.timeloop_style_even = True
    pmappings = ff.mapper.FFM.make_pmappings(spec)
    pmappings = pmappings.filter(no_hybrid_mapping_filter)
    tileflow_mappings = ff.mapper.FFM.join_pmappings(spec, pmappings)

    return loopforest_mappings, looptree_mappings, tileflow_mappings, unfused_mappings

In [ ]:
def compute_latency(all_mappings, names, baseline, fname):
    for name, mappings in zip(names, all_mappings):
        if name != baseline:
            continue
        baseline_latency = mappings.data["Total<SEP>latency"].iloc[0]
        baseline_energy = mappings.data["Total<SEP>energy"].iloc[0]
        ideal_latency = mappings.data[[c for c in mappings.data.columns if "latency" in c and "compute" in c]].iloc[0].sum()

    message = ''
    for name, mappings in zip(names, all_mappings):
        message += name + '\n'
        latency = mappings.data["Total<SEP>latency"].iloc[0]
        energy = mappings.data["Total<SEP>energy"].iloc[0]
        message += f"{latency/baseline_latency} ({latency/ideal_latency}); {energy/baseline_energy}\n"
    
    with open(fname, "w") as f:
        f.write(message)

In [ ]:
for shape in [7, 14, 28]:
    lf, lt, tf, un = run_experiments(shape)
    compute_latency([lf, lt, tf, un], ["LoopForest", 'LoopTree', 'TileFlow', 'Unfused'], 'LoopForest', f"result_{shape}.txt")

WARNING Loading configuration file from /home/gilbertm/work/venv/fastfusion/config.yaml
Generating pmapping templates for compute MAC Einsum PwiseA0: 24it [00:00, 84.18it/s]
Generating pmapping templates for compute MAC Einsum PwiseB2: 24it [00:00, 84.25it/s]
Generating pmapping templates for compute MAC Einsum PwiseB0: 48it [00:00, 92.62it/s] 
Generating pmapping templates for compute MAC Einsum PwiseA1: 48it [00:00, 96.92it/s] 
Generating pmapping templates for compute MAC Einsum Dwise0: 252it [00:01, 151.59it/s]
Generating pmapping templates for compute MAC Einsum Dwise1: 252it [00:01, 149.50it/s]
Generating jobs: 100%|██████████| 6/6 [00:04<00:00,  1.43it/s]
WARNING Einsum PwiseA0 has 24 pmapping templates:
WARNING 	[WA0,T0 in MainMemory] T-p0,q0,t0  [T0 in GlobalBuffer] T-t0  [TA0 in GlobalBuffer] T-n0,t0  [WA0 in GlobalBuffer] T-p0,q0  S-Y-n0  S-X-t0  MAC computes PwiseA0
WARNING 	[WA0,T0 in MainMemory] T-p0,q0,t0  [T0 in GlobalBuffer] T-t0  [TA0 in GlobalBuffer] T-n0,t0  [WA0 in

PwiseA0: 2.43e06 total, 1.50e06 (1/2) valid, 9.26e04 (1/26) evaluated, 4.31e03 (1/565) Pareto-Optimal
Dwise0: 3.24e06 total, 1.67e06 (1/2) valid, 3.42e06 (1/1) evaluated, 2.62e04 (1/124) Pareto-Optimal
PwiseB0: 3.12e06 total, 2.01e06 (1/2) valid, 1.42e05 (1/22) evaluated, 6.58e03 (1/473) Pareto-Optimal
PwiseA1: 3.37e06 total, 2.06e06 (1/2) valid, 1.41e05 (1/24) evaluated, 6.67e03 (1/505) Pareto-Optimal
Dwise1: 1.77e06 total, 9.75e05 (1/2) valid, 2.47e06 (1/1) evaluated, 1.09e04 (1/163) Pareto-Optimal
PwiseB2: 2.56e06 total, 1.54e06 (1/2) valid, 9.64e04 (1/27) evaluated, 4.16e03 (1/615) Pareto-Optimal
Total: 1.65e07 total, 9.76e06 (1/2) valid, 6.36e06 (1/3) evaluated, 5.88e04 (1/280) Pareto-Optimal
Einsum PwiseA0 has 4309 pmappings with 37 compatibilities
Einsum Dwise0 has 26245 pmappings with 178 compatibilities
Einsum PwiseB0 has 6582 pmappings with 129 compatibilities
Einsum PwiseA1 has 6672 pmappings with 129 compatibilities
Einsum Dwise1 has 10879 pmappings with 84 compatibilities


Compressing pmappings: 100%|██████████| 6/6 [00:00<00:00, 12.34it/s]


Not tracking MainMemory because it is never reserved for multiple pmappings.
Not tracking Register because it is never reserved for multiple pmappings.


Merging pmappings for Dwise1 <--> PwiseB2 (6/6): 100%|██████████| 86/86 [00:02<00:00, 31.79it/s] 
/home/gilbertm/work/active/fastfusion/fastfusion/mapper/FFM/_interface/main.py:162: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  joined._data = joined.data.fillna(0).reset_index(drop=True)


...

Final consolidate: 100%|██████████| 1/1 [00:00<00:00, 27235.74it/s]
Generating pmapping templates for compute MAC Einsum PwiseA0: 24it [00:00, 46.22it/s]
Generating pmapping templates for compute MAC Einsum PwiseB2: 24it [00:00, 43.74it/s]
Generating pmapping templates for compute MAC Einsum PwiseB0: 48it [00:00, 57.22it/s]
Generating pmapping templates for compute MAC Einsum PwiseA1: 48it [00:00, 50.88it/s]
Generating pmapping templates for compute MAC Einsum Dwise0: 252it [00:03, 74.88it/s] 
Generating pmapping templates for compute MAC Einsum Dwise1: 252it [00:03, 73.57it/s] 
Generating jobs: 100%|██████████| 6/6 [00:06<00:00,  1.06s/it]
WARNING Einsum PwiseA0 has 24 pmapping templates:
WARNING 	[WA0,T0 in MainMemory] T-p0,q0,t0  [T0 in GlobalBuffer] T-t0  [TA0,WA0 in GlobalBuffer] T-n0,p0,q0,t0  S-Y-n0  S-X-t0  MAC computes PwiseA0
WARNING 	[WA0,T0 in MainMemory] T-p0,q0,t0  [T0 in GlobalBuffer] T-t0  [WA0,TA0 in GlobalBuffer] T-n0,p0,q0,t0  S-Y-n0  S-X-t0  MAC computes PwiseA0
WAR

Dwise0: 3.31e06 total, 1.95e06 (1/2) valid, 2.30e06 (1/1) evaluated, 4.95e03 (1/669) Pareto-Optimal
Dwise1: 1.94e06 total, 1.20e06 (1/2) valid, 1.58e06 (1/1) evaluated, 2.45e03 (1/792) Pareto-Optimal
PwiseA0: 2.54e06 total, 1.68e06 (1/2) valid, 7.81e04 (1/33) evaluated, 1.20e03 (1/2.12e03) Pareto-Optimal
PwiseB0: 3.22e06 total, 2.22e06 (1/1) valid, 1.10e05 (1/29) evaluated, 1.96e03 (1/1.64e03) Pareto-Optimal
PwiseA1: 3.48e06 total, 2.26e06 (1/2) valid, 1.10e05 (1/32) evaluated, 1.90e03 (1/1.83e03) Pareto-Optimal
PwiseB2: 2.68e06 total, 1.67e06 (1/2) valid, 8.22e04 (1/33) evaluated, 1.25e03 (1/2.15e03) Pareto-Optimal
Total: 1.72e07 total, 1.10e07 (1/2) valid, 4.26e06 (1/4) evaluated, 1.37e04 (1/1.25e03) Pareto-Optimal
Einsum PwiseA0 has 1195 pmappings with 23 compatibilities
Einsum Dwise0 has 2676 pmappings with 34 compatibilities
Einsum PwiseB0 has 1609 pmappings with 51 compatibilities
Einsum PwiseA1 has 1548 pmappings with 51 compatibilities
Einsum Dwise1 has 1339 pmappings with 20 c

Compressing pmappings: 100%|██████████| 6/6 [00:00<00:00, 78.38it/s]


Not tracking MainMemory because it is never reserved for multiple pmappings.
Not tracking Register because it is never reserved for multiple pmappings.


Merging pmappings for Dwise1 <--> PwiseB2 (6/6): 100%|██████████| 51/51 [00:00<00:00, 689.78it/s]
